In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
import numpy as np
# from tensorflow_model_optimization.python.core.keras.compat import keras
from tensorflow import keras
import matplotlib.pyplot as plt
import random
tf.random.set_seed(3407)
np.random.seed(3407)
random.seed(3407)

from create_spectrogram_and_time_series_dataset_use import (
    create_spectrograms_from_audio_dataset, 
)
from helper_functions import (
    get_file_size, 
    convert_bytes, 
    convert_prefetchdataset_to_numpy_arrays,
    predict_and_print_full_results,
    evaluate_time_of_prediction
    )

2024-06-18 19:17:32.921258: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-18 19:17:33.012685: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-18 19:17:33.012730: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-18 19:17:33.016513: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-18 19:17:33.031472: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-18 19:17:33.032943: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [3]:
sample_rate = 16000

In [4]:
train_dataset = tf.keras.utils.audio_dataset_from_directory("dataset/training", labels='inferred', sampling_rate=sample_rate, batch_size=32, shuffle=True, seed=3407)
test_dataset = tf.keras.utils.audio_dataset_from_directory("dataset/testing", labels='inferred', sampling_rate=sample_rate, batch_size=32, shuffle=True, seed=3407)
val_dataset = tf.keras.utils.audio_dataset_from_directory("dataset/validation", labels='inferred', sampling_rate=sample_rate, batch_size=32, shuffle=True, seed=3407)

label_names = np.array(test_dataset.class_names)
print("Classes: ", label_names)

Found 11292 files belonging to 2 classes.


2024-06-18 19:17:38.487677: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-18 19:17:38.488198: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-06-18 19:17:38.616321: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
2024-06-18 19:17:38.624138: W tensorflow_io/core/kernels/audio_video_mp3_kernels.cc:271] libmp3lame.so.0 or lame functions are not available


Found 1393 files belonging to 2 classes.
Found 1380 files belonging to 2 classes.
Classes:  ['non_target' 'target']


In [5]:
train_spectrogram_ds = create_spectrograms_from_audio_dataset(train_dataset, sample_rate = sample_rate).cache().prefetch(tf.data.AUTOTUNE)
test_spectrogram_ds = create_spectrograms_from_audio_dataset(test_dataset, sample_rate = sample_rate).cache().prefetch(tf.data.AUTOTUNE)
val_spectrogram_ds = create_spectrograms_from_audio_dataset(val_dataset, sample_rate = sample_rate).cache().prefetch(tf.data.AUTOTUNE)

x_train_np, y_train_np = convert_prefetchdataset_to_numpy_arrays(train_spectrogram_ds)
x_val_np, y_val_np = convert_prefetchdataset_to_numpy_arrays(val_spectrogram_ds)
x_test_np, y_test_np = convert_prefetchdataset_to_numpy_arrays(test_spectrogram_ds)

In [6]:
BASE_MODEL_FILE_NAME = "spectrogram_models_from_notebooks/squeezenet/squeezenet_spec_16kHz_baseline.keras"
baseline_model = keras.models.load_model(BASE_MODEL_FILE_NAME)
baseline_model.summary()
convert_bytes(get_file_size(BASE_MODEL_FILE_NAME), "KB")

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 184, 80, 1)]         0         []                            
                                                                                                  
 conv1 (Conv2D)              (None, 92, 40, 96)           4800      ['input_1[0][0]']             
                                                                                                  
 maxpool1 (MaxPooling2D)     (None, 45, 19, 96)           0         ['conv1[0][0]']               
                                                                                                  
 fire2_squeeze (Conv2D)      (None, 45, 19, 16)           1552      ['maxpool1[0][0]']            
                                                                                              

### Convert to TF Lite format

In [7]:
converter = tf.lite.TFLiteConverter.from_keras_model(baseline_model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp61xtrsof/assets


INFO:tensorflow:Assets written to: /tmp/tmp61xtrsof/assets
2024-06-18 19:18:39.762187: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-06-18 19:18:39.762239: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-06-18 19:18:39.765524: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp61xtrsof
2024-06-18 19:18:39.771556: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-06-18 19:18:39.771593: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmp61xtrsof
2024-06-18 19:18:39.786992: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-06-18 19:18:39.791589: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-06-18 19:18:40.033481: I tensorflow/cc/saved_model/loader.cc:217] Running initializatio

In [9]:
print("Validation dataset:")
(
    y_pred_val, 
    non_overlap_patritions_f1_scores_val, 
    bootstrap_patritions_f1_scores_val,
    ) = predict_and_print_full_results(tflite_model, x_val_np, y_val_np, model_format="tf_lite")

print("\nTest dataset:")
(
    y_pred_test, 
    non_overlap_patritions_f1_scores_test, 
    bootstrap_patritions_f1_scores_test,
    ) = predict_and_print_full_results(tflite_model, x_test_np, y_test_np, model_format="tf_lite")

print("\nTime for Test dataset:")
evaluate_time_of_prediction(tflite_model, x_test_np, y_test_np, model_format="tf_lite")

TF_LITE_MODEL_FILE_NAME = "spectrogram_models_from_notebooks/squeezenet/squeezenet_spec_16kHz.tflite"
open(TF_LITE_MODEL_FILE_NAME, "wb").write(tflite_model)
print("\n")
print("Model file name: ", TF_LITE_MODEL_FILE_NAME)
convert_bytes(get_file_size(TF_LITE_MODEL_FILE_NAME), "MB")

Validation dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 98.77%
Recall: 99.36%
Precision: 97.10%
F1-score: 98.22%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9821834731737121

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.982104204804031

Test dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 98.56%
Recall: 99.56%
Precision: 96.20%
F1-score: 97.85%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9778618945971255

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9816260736721216

Time for Test dataset:
Accuracy: 98.56%
Recall: 99.56%
Precision: 96.20%
F1-score: 97.85%

Time to make a prediction for a single data point
Mean: 0.008 seconds
Max: 0.018 seconds
M

In [10]:
convert_bytes(get_file_size(TF_LITE_MODEL_FILE_NAME), "KB")

File size: 2856.07 Kilobytes


### Dynamic range quantization

In [11]:
dynamic_range_quant_converter = tf.lite.TFLiteConverter.from_keras_model(baseline_model)
dynamic_range_quant_converter.optimizations = [tf.lite.Optimize.DEFAULT]
dynamic_range_quant_model = dynamic_range_quant_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpq0h4ue_5/assets


INFO:tensorflow:Assets written to: /tmp/tmpq0h4ue_5/assets
2024-06-18 19:24:10.167516: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-06-18 19:24:10.167616: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-06-18 19:24:10.168030: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpq0h4ue_5
2024-06-18 19:24:10.182698: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-06-18 19:24:10.182738: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpq0h4ue_5
2024-06-18 19:24:10.251176: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-06-18 19:24:10.582257: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmpq0h4ue_5
2024-06-18 19:24:10.657816: I tensorflow/cc/saved_model/loader.cc:316] SavedModel

In [12]:
print("Validation dataset:")
(
    y_pred_val, 
    non_overlap_patritions_f1_scores_val, 
    bootstrap_patritions_f1_scores_val,
    ) = predict_and_print_full_results(dynamic_range_quant_model, x_val_np, y_val_np, model_format="tf_lite")

print("\nTest dataset:")
(
    y_pred_test, 
    non_overlap_patritions_f1_scores_test, 
    bootstrap_patritions_f1_scores_test,
    ) = predict_and_print_full_results(dynamic_range_quant_model, x_test_np, y_test_np, model_format="tf_lite")

print("\nTime for Test dataset:")
evaluate_time_of_prediction(dynamic_range_quant_model, x_test_np, y_test_np, model_format="tf_lite")

DRQ_MODEL_FILE_NAME = "spectrogram_models_from_notebooks/squeezenet/squeezenet_spec_16kHz_drq.tflite"
open(DRQ_MODEL_FILE_NAME, "wb").write(dynamic_range_quant_model)
print("\n")
print("Model file name: ", DRQ_MODEL_FILE_NAME)
convert_bytes(get_file_size(DRQ_MODEL_FILE_NAME), "KB")

Validation dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 98.84%
Recall: 99.36%
Precision: 97.30%
F1-score: 98.32%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9831180526129645

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9842558601523974

Test dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 98.56%
Recall: 99.56%
Precision: 96.20%
F1-score: 97.85%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9778618945971255

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9754389446688596

Time for Test dataset:
Accuracy: 98.56%
Recall: 99.56%
Precision: 96.20%
F1-score: 97.85%

Time to make a prediction for a single data point
Mean: 0.012 seconds
Max: 0.021 seconds


### Float16 quantization

In [13]:
float16_quant_converter = tf.lite.TFLiteConverter.from_keras_model(baseline_model)
float16_quant_converter.optimizations = [tf.lite.Optimize.DEFAULT]
float16_quant_converter.target_spec.supported_types = [tf.float16]
float16_quant_model = float16_quant_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpi6b2dlwj/assets


INFO:tensorflow:Assets written to: /tmp/tmpi6b2dlwj/assets
2024-06-18 19:29:07.792832: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-06-18 19:29:07.792886: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-06-18 19:29:07.793060: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpi6b2dlwj
2024-06-18 19:29:07.813358: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-06-18 19:29:07.813392: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpi6b2dlwj
2024-06-18 19:29:07.858908: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-06-18 19:29:08.056527: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmpi6b2dlwj
2024-06-18 19:29:08.111195: I tensorflow/cc/saved_model/loader.cc:316] SavedModel

In [14]:
print("Validation dataset:")
(
    y_pred_val, 
    non_overlap_patritions_f1_scores_val, 
    bootstrap_patritions_f1_scores_val,
    ) = predict_and_print_full_results(float16_quant_model, x_val_np, y_val_np, model_format="tf_lite")

print("\nTest dataset:")
(
    y_pred_test, 
    non_overlap_patritions_f1_scores_test, 
    bootstrap_patritions_f1_scores_test,
    ) = predict_and_print_full_results(float16_quant_model, x_test_np, y_test_np, model_format="tf_lite")

print("\nTime for Test dataset:")
evaluate_time_of_prediction(float16_quant_model, x_test_np, y_test_np, model_format="tf_lite")

FLOAT16_QUANT_MODEL_FILE_NAME = "spectrogram_models_from_notebooks/squeezenet/squeezenet_spec_16kHz_float16q.tflite"
open(FLOAT16_QUANT_MODEL_FILE_NAME, "wb").write(float16_quant_model)
print("\n")
print("Model file name: ", FLOAT16_QUANT_MODEL_FILE_NAME)
convert_bytes(get_file_size(FLOAT16_QUANT_MODEL_FILE_NAME), "MB")

Validation dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 98.77%
Recall: 99.36%
Precision: 97.10%
F1-score: 98.22%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9821834731737121

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9822585899607365

Test dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 98.56%
Recall: 99.56%
Precision: 96.20%
F1-score: 97.85%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9778618945971255

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.974594660289636

Time for Test dataset:
Accuracy: 98.56%
Recall: 99.56%
Precision: 96.20%
F1-score: 97.85%

Time to make a prediction for a single data point
Mean: 0.009 seconds
Max: 0.018 seconds
M

In [15]:
convert_bytes(get_file_size(FLOAT16_QUANT_MODEL_FILE_NAME), "KB")

File size: 1442.977 Kilobytes


### Float fallback quantization

In [23]:
float_fallback_quant_converter = tf.lite.TFLiteConverter.from_keras_model(baseline_model)
float_fallback_quant_converter.optimizations = [tf.lite.Optimize.DEFAULT]
float_fallback_quant_converter.representative_dataset = representative_data_gen
float_fallback_quant_model = float_fallback_quant_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpupnb9sup/assets


INFO:tensorflow:Assets written to: /tmp/tmpupnb9sup/assets
/home/polina/test/venv/lib/python3.9/site-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2024-06-18 19:35:39.474812: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-06-18 19:35:39.474913: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-06-18 19:35:39.475081: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpupnb9sup
2024-06-18 19:35:39.480159: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-06-18 19:35:39.480183: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpupnb9sup
2024-06-18 19:35:39.502694: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-06-18 19:35:39.

In [24]:
print("Validation dataset:")
(
    y_pred_val, 
    non_overlap_patritions_f1_scores_val, 
    bootstrap_patritions_f1_scores_val,
    ) = predict_and_print_full_results(float_fallback_quant_model, x_val_np, y_val_np, model_format="tf_lite")

print("\nTest dataset:")
(
    y_pred_test, 
    non_overlap_patritions_f1_scores_test, 
    bootstrap_patritions_f1_scores_test,
    ) = predict_and_print_full_results(float_fallback_quant_model, x_test_np, y_test_np, model_format="tf_lite")

print("\nTime for Test dataset:")
evaluate_time_of_prediction(float_fallback_quant_model, x_test_np, y_test_np, model_format="tf_lite")

FLOAT_FALLBACK_QUANT_MODEL_FILE_NAME = "spectrogram_models_from_notebooks/squeezenet/squeezenet_spec_16kHz_float_fallback_q.tflite"
open(FLOAT_FALLBACK_QUANT_MODEL_FILE_NAME, "wb").write(float_fallback_quant_model)
print("\n")
print("Model file name: ", FLOAT_FALLBACK_QUANT_MODEL_FILE_NAME)
convert_bytes(get_file_size(FLOAT_FALLBACK_QUANT_MODEL_FILE_NAME), "MB")

Validation dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 98.33%
Recall: 96.82%
Precision: 98.28%
F1-score: 97.55%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9749951630388505

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9760731261677763

Test dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 98.21%
Recall: 97.82%
Precision: 96.76%
F1-score: 97.29%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9717964547130997

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.971747845629563

Time for Test dataset:
Accuracy: 98.21%
Recall: 97.82%
Precision: 96.76%
F1-score: 97.29%

Time to make a prediction for a single data point
Mean: 0.01 seconds
Max: 0.025 seconds
Mi

In [25]:
convert_bytes(get_file_size(FLOAT_FALLBACK_QUANT_MODEL_FILE_NAME), "KB")

File size: 807.516 Kilobytes
